# Importar banco de dados

Exemplo de criação de API usando o Pycaret (predição HeartDisease)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    'heart.csv'
)
df.head()

Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


Não existe presença de NaN's nesse data-frame

y - HeartDisease

entradas - demais variáveis.

In [3]:
df.HeartDisease.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [4]:
import plotly.express as px

In [5]:
px.histogram(
    df,
    x='RestingBP',
    title= 'Histograma da RestingBP.'
)

In [6]:
px.box(
    df,
    x='RestingBP',
    title= 'Histograma da RestingBP.'
)

In [7]:
px.histogram(
    df,
    x='Age',
    title= 'Histograma da idade.'
)

In [8]:
px.histogram(
    df,
    x='Cholesterol',
    title= 'Histograma do Cholesterol.'
)

In [9]:
df.describe()

Age   RestingBP  Cholesterol   FastingBS       MaxHR  \
count  918.000000  918.000000   918.000000  918.000000  918.000000   
mean    53.510893  132.396514   198.799564    0.233115  136.809368   
std      9.432617   18.514154   109.384145    0.423046   25.460334   
min     28.000000    0.000000     0.000000    0.000000   60.000000   
25%     47.000000  120.000000   173.250000    0.000000  120.000000   
50%     54.000000  130.000000   223.000000    0.000000  138.000000   
75%     60.000000  140.000000   267.000000    0.000000  156.000000   
max     77.000000  200.000000   603.000000    1.000000  202.000000   

          Oldpeak  HeartDisease  
count  918.000000    918.000000  
mean     0.887364      0.553377  
std      1.066570      0.497414  
min     -2.600000      0.000000  
25%      0.000000      0.000000  
50%      0.600000      1.000000  
75%      1.500000      1.000000  
max      6.200000      1.000000

Cholesterol e RestingBP iguais a zero seriam dados estranhos.

Colocarei os dados que são zero do colesterol como sendo a mediana.

In [10]:
df['Cholesterol'] = df['Cholesterol'].apply(
    lambda x: df.Cholesterol.median() if x <10 else x
    )
df['RestingBP'] = df['RestingBP'].apply(
    lambda x: df.RestingBP.median() if x <10 else x
    )
df.describe()

Age   RestingBP  Cholesterol   FastingBS       MaxHR  \
count  918.000000  918.000000   918.000000  918.000000  918.000000   
mean    53.510893  132.538126   240.581699    0.233115  136.809368   
std      9.432617   17.990127    53.982967    0.423046   25.460334   
min     28.000000   80.000000    85.000000    0.000000   60.000000   
25%     47.000000  120.000000   214.000000    0.000000  120.000000   
50%     54.000000  130.000000   223.000000    0.000000  138.000000   
75%     60.000000  140.000000   267.000000    0.000000  156.000000   
max     77.000000  200.000000   603.000000    1.000000  202.000000   

          Oldpeak  HeartDisease  
count  918.000000    918.000000  
mean     0.887364      0.553377  
std      1.066570      0.497414  
min     -2.600000      0.000000  
25%      0.000000      0.000000  
50%      0.600000      1.000000  
75%      1.500000      1.000000  
max      6.200000      1.000000

In [11]:
px.histogram(
    df,
    x='Oldpeak',
    title= 'Histograma do Oldpeak.'
)

# Usando a biblioteca Pycaret

In [12]:
from pycaret.classification import *

In [13]:
modelo_setup = setup(
    df,
    target='HeartDisease',
    remove_outliers=True,
    remove_multicollinearity=True,
    remove_perfect_collinearity=True    
)

In [14]:
modelo = compare_models()

In [15]:
tunar_modelo = tune_model(modelo)

In [18]:
evaluate_model(tunar_modelo)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [19]:
tunar_modelo.save_model('modelo_tunado')

In [20]:
create_api(tunar_modelo,api_name='heartprediction')


API sucessfully created. This function only creates a POST API, it doesn't run it automatically.

To run your API, please run this command --> !python heartprediction.py
    


In [21]:
create_app(tunar_modelo)

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1fb617756a0>, 'http://127.0.0.1:7860/', None)